<a href="https://colab.research.google.com/github/logsmay/inter-probe/blob/main/Apache_Beam_Colab_LZD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run and print a shell command.
def run(cmd):
  print('>> {}'.format(cmd))
  !{cmd}
  print('')

# Install apache-beam.
run('pip install --quiet apache-beam')

>> pip install --quiet apache-beam
     |████████████████████████████████| 3.0MB 9.2MB/s 
     |████████████████████████████████| 235kB 57.0MB/s 
     |████████████████████████████████| 153kB 60.3MB/s 
     |████████████████████████████████| 51kB 3.1MB/s 
     |████████████████████████████████| 1.2MB 54.6MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 58.1MB 37kB/s 
     |████████████████████████████████| 112kB 40.7MB/s 
ERROR: pydrive 1.3.1 has requirement oauth2client>=4.0.0, but you'll have oauth2client 3.0.0 which is incompatible.
ERROR: multiprocess 0.70.9 has requirement dill>=0.3.1, but you'll have dill 0.3.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.



[Apache Beam](https://beam.apache.org/) is an unified programming SDK to design and execute data processing pipelines (both batch & realtime) with multiple supported runners such as [Apache Flink](https://flink.apache.org/), [Apache Spark](https://spark.apache.org/), [Apache Apex](https://apex.apache.org/), [Apache Gearpump](http://incubator.apache.org/projects/gearpump.html), [Apache Samza](http://samza.apache.org/) and [Google Dataflow](https://cloud.google.com/dataflow/)

In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.coders.coders import Coder
import json
import logging
from datetime import datetime

In [ ]:
logging.getLogger().setLevel(logging.INFO)

Apache Beam supports [multiple variants of runners](https://beam.apache.org/documentation/runners/capability-matrix/) to build and process data pipelines through highly sophisticated parallel processing. Each runners can be configured accordingly with the respective configuration through pipeline options.

In [ ]:
# Current option uses direct runner (local python kernel)
option_dict = {
    'runner': 'DirectRunner',
    'job_name': 'notebook',
}
pipeline_options = PipelineOptions.from_dictionary(option_dict)

In [ ]:
input_data = [
    "1b,20181019,10:00 am,click,ios,SG",
    "1b,20181019,10:01 am,install,ios,SG",
    "1c,20181019,10:02 am,page_view,android,SG",
    "1c,20181019,10:03 am,order,android,SG",
    "1b,20181019,11:00 am,page_view,ios,SG",
    "1b,20181019,11:10 am,order,ios,SG",
    "1c,20181019,12:00 pm,page_view,andorid,SG"
]
outputs_prefix = 'outputs/part'

In [ ]:
session_gap =  (60 * 60) - 60 

print("removing old files")
run('rm -rf {}*'.format("outputs/"))

# class to prepare data before processing
# add preprocessed session id field & formatted timestamp field from date & time
class DataPrep(beam.DoFn):
    def process(self, element):
        pcol = element.split(",")
        ts_concat = pcol[1] + " " + pcol[2]
        session_id_ = 's' + pcol[0] + "_" 
        session_ts = datetime.strptime(ts_concat, '%Y%m%d %I:%M %p').timestamp()
        pcol.append(session_id_)
        pcol.append(session_ts)
        return [pcol]

# class to add processed session id based on calculated window parameter 
class AddSessionId(beam.DoFn):
  def process(self, element, window=beam.DoFn.WindowParam):
    for x in element[1]:
      x[6] += window.start.to_utc_datetime().strftime("%Y%m%d") + "_" + window.start.to_utc_datetime().strftime("%H%M")
      logging.info(" Event >> %s %s %s with start window=%s"\
                   , x[0], x[3], x[7], window.start.to_utc_datetime())
      yield x[:-1]

# class to export file by session id
class ExportFileBySession(beam.DoFn):
    def __init__(self):
        self.outdir = 'outputs/out-s1h_'
    def process(self, element):
        from apache_beam.io.filesystems import FileSystems 
        print("writing file: " + element[0])
        exp_file_name = self.outdir+element[0]+'.csv'
        writer = FileSystems.create(exp_file_name, 'text/plain')
        session_str = ""
        for row in element[1]:
          session_str += ','.join(row) + '\n'
        writer.write(session_str.encode())
        writer.close()

# apache beam processing starts here with defining pipeline object with options
ptemp = beam.Pipeline(options=pipeline_options)

# start processing files through series of PCollection processing (similar to spark)
pline = (
    ptemp
    | 'loaddata' >> beam.Create(input_data) # can include variety of input using beam.io (batch/realtime)
    | 'prepdata' >> beam.ParDo(DataPrep())
    | 'addts' >> beam.Map(lambda elem: beam.window.TimestampedValue(elem, elem[7]))
    | 'addkey' >> beam.Map(lambda row: (row[0], row))
    | 'windowtimer' >> beam.WindowInto(beam.window.Sessions(session_gap)\
                                       , timestamp_combiner=beam.window.TimestampCombiner.OUTPUT_AT_EOW)
    | 'groupbydevice' >> beam.GroupByKey()
    | 'addsession' >> beam.ParDo(AddSessionId())
    | 'prepexport' >> beam.Map(lambda row: (row[6], row))
    | 'groupbysessionid' >> beam.GroupByKey()
)

# PCollection - just to print on console
(
    pline
    | 'print' >> beam.Map(lambda element: print(element))
)

# PCollection - to export file by generated session id
(
    pline
    | 'exportsession' >> beam.ParDo(ExportFileBySession())
)

result = ptemp.run()
result.wait_until_finish()

removing old files
>> rm -rf outputs/*



INFO:root:==================== <function annotate_downstream_side_inputs at 0x7fb695a33ae8> ====================
INFO:root:==================== <function fix_side_input_pcoll_coders at 0x7fb695a33bf8> ====================
INFO:root:==================== <function lift_combiners at 0x7fb695a33c80> ====================
INFO:root:==================== <function expand_sdf at 0x7fb695a33d08> ====================
INFO:root:==================== <function expand_gbk at 0x7fb695a33d90> ====================
INFO:root:==================== <function sink_flattens at 0x7fb695a33ea0> ====================
INFO:root:==================== <function greedily_fuse at 0x7fb695a33f28> ====================
INFO:root:==================== <function read_to_impulse at 0x7fb695a44048> ====================
INFO:root:==================== <function impulse_to_input at 0x7fb695a440d0> ====================
INFO:root:==================== <function inject_timer_pcollections at 0x7fb695a44268> ====================
INFO:r

('s1b_20181019_1000', [['1b', '20181019', '10:00 am', 'click', 'ios', 'SG', 's1b_20181019_1000'], ['1b', '20181019', '10:01 am', 'install', 'ios', 'SG', 's1b_20181019_1000']])
writing file: s1b_20181019_1000
('s1b_20181019_1100', [['1b', '20181019', '11:00 am', 'page_view', 'ios', 'SG', 's1b_20181019_1100'], ['1b', '20181019', '11:10 am', 'order', 'ios', 'SG', 's1b_20181019_1100']])
writing file: s1b_20181019_1100
('s1c_20181019_1002', [['1c', '20181019', '10:02 am', 'page_view', 'android', 'SG', 's1c_20181019_1002'], ['1c', '20181019', '10:03 am', 'order', 'android', 'SG', 's1c_20181019_1002']])
writing file: s1c_20181019_1002
('s1c_20181019_1200', [['1c', '20181019', '12:00 pm', 'page_view', 'andorid', 'SG', 's1c_20181019_1200']])
writing file: s1c_20181019_1200


'DONE'

In [ ]:
run('head -n 20 {}*'.format("outputs/"))

>> head -n 20 outputs/*
==> outputs/out-s1b_20181019_1000.csv <==
1b,20181019,10:00 am,click,ios,SG,s1b_20181019_1000
1b,20181019,10:01 am,install,ios,SG,s1b_20181019_1000

==> outputs/out-s1b_20181019_1100.csv <==
1b,20181019,11:00 am,page_view,ios,SG,s1b_20181019_1100
1b,20181019,11:10 am,order,ios,SG,s1b_20181019_1100

==> outputs/out-s1c_20181019_1002.csv <==
1c,20181019,10:02 am,page_view,android,SG,s1c_20181019_1002
1c,20181019,10:03 am,order,android,SG,s1c_20181019_1002

==> outputs/out-s1c_20181019_1200.csv <==
1c,20181019,12:00 pm,page_view,andorid,SG,s1c_20181019_1200

